In [1]:
!pip install wfdb pandas matplotlib scikit-learn


  Attempting uninstall: joblib
    Found existing installation: joblib 1.1.1
    Uninstalling joblib-1.1.1:
      Successfully uninstalled joblib-1.1.1


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pandas-profiling 3.2.0 requires joblib~=1.1.0, but you have joblib 1.5.1 which is incompatible.


In [3]:
import pandas as pd
df = pd.read_csv('Dataset/ptbxl/ptbxl_database.csv')
df.head(5)

,ecg_id,patient_id,age,sex,height,weight,nurse,site,device,recording_date,...,validated_by_human,baseline_drift,static_noise,burst_noise,electrodes_problems,extra_beats,pacemaker,strat_fold,filename_lr,filename_hr
0,1,15709.0,56.0,1,NaN,63.0,2.0,0.0,CS-12 E,1984-11-09 09:17:34,...,True,NaN,", I-V1,",NaN,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr
1,2,13243.0,19.0,0,NaN,70.0,2.0,0.0,CS-12 E,1984-11-14 12:55:37,...,True,NaN,NaN,NaN,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr
2,3,20372.0,37.0,1,NaN,69.0,2.0,0.0,CS-12 E,1984-11-15 12:49:10,...,True,NaN,NaN,NaN,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr
3,4,17014.0,24.0,0,NaN,82.0,2.0,0.0,CS-12 E,1984-11-15 13:44:57,...,True,", II,III,AVF",NaN,NaN,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr
4,5,17448.0,19.0,1,NaN,70.0,2.0,0.0,CS-12 E,1984-11-17 10:43:15,...,True,", III,AVR,AVF",NaN,NaN,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr


In [ ]:
import ast

df['scp_codes'] = df['scp_codes'].apply(ast.literal_eval)
# Convert the 'scp_codes' column to a list of dictionaries
df['scp_keys'] = df['scp_codes'].apply(lambda x: list(x.keys()))
# Convert the 'scp_codes' column to a list of lists


In [5]:
target_diseases = {
    "has_MI": ["MI", "IMI"],
    "has_CD": ["CD"],
    "has_ISCA": ["ISCA"],
}

In [6]:
for target_col, scp_list in target_diseases.items():
    df[target_col] = df["scp_codes"].apply(lambda x: any(code in x for code in scp_list))

In [7]:
for col in ["age", "height", "weight"]:
    df[col].fillna(df[col].median(), inplace=True)

C:\Users\estya\AppData\Local\Temp\ipykernel_12688\3214849954.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[col].fillna(df[col].median(), inplace=True)


In [9]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df[["age", "height", "weight"]] = scaler.fit_transform(df[["age", "height", "weight"]])

In [10]:
drop_cols = [
    "recording_date", "report", "nurse", "doctor",
    "filename_lr", "filename_hr", "scp_codes",
    "baseline_drift", "burst_noise", "electrodes_problems",
    "extra_beats", "movement_artifact"
]

In [11]:
df_cleaned = df.drop(columns=[col for col in drop_cols if col in df.columns])

In [12]:
df_cleaned.head(5)

,ecg_id,patient_id,age,sex,height,weight,site,device,heart_axis,infarction_stadium1,...,second_opinion,initial_autogenerated_report,validated_by_human,static_noise,pacemaker,strat_fold,scp_keys,has_MI,has_CD,has_ISCA
0,1,15709.0,-0.227271,1,-0.036814,-0.711564,0.0,CS-12 E,NaN,NaN,...,False,False,True,", I-V1,",NaN,3,"[NORM, LVOLT, SR]",False,False,False
1,2,13243.0,-2.414198,0,-0.036814,-0.041247,0.0,CS-12 E,NaN,NaN,...,False,False,True,NaN,NaN,2,"[NORM, SBRAD]",False,False,False
2,3,20372.0,-1.350287,1,-0.036814,-0.137007,0.0,CS-12 E,NaN,NaN,...,False,False,True,NaN,NaN,5,"[NORM, SR]",False,False,False
3,4,17014.0,-2.118667,0,-0.036814,1.107868,0.0,CS-12 E,NaN,NaN,...,False,False,True,NaN,NaN,3,"[NORM, SR]",False,False,False
4,5,17448.0,-2.414198,1,-0.036814,-0.041247,0.0,CS-12 E,NaN,NaN,...,False,False,True,NaN,NaN,4,"[NORM, SR]",False,False,False


In [13]:
df_cleaned.to_csv("ptbxl_cleaned_metadata.csv", index=False)

In [14]:
df.drop(columns=["heart_axis", "infarction_stadium2", "pacemaker"], inplace=True)


In [16]:
print(df["has_MI"].value_counts())  # Check imbalance


has_MI
False    19152
True      2685
Name: count, dtype: int64


In [17]:
df.drop(columns=["ecg_id", "patient_id"], inplace=True)


In [18]:
print(df.isnull().sum())   # Check if anything still missing
print(df.dtypes)           # Ensure all types are ML-ready
print(df.shape)            # Final dataset size


age                                 0
sex                                 0
height                              0
weight                              0
nurse                            1509
site                               18
device                              0
recording_date                      0
report                              0
scp_codes                           0
infarction_stadium1             16211
validated_by                     9411
second_opinion                      0
initial_autogenerated_report        0
validated_by_human                  0
baseline_drift                  20230
static_noise                    18575
burst_noise                     21224
electrodes_problems             21807
extra_beats                     19883
strat_fold                          0
filename_lr                         0
filename_hr                         0
scp_keys                            0
has_MI                              0
has_CD                              0
has_ISCA    

In [19]:
df.head(5)  # Display the first few rows of the cleaned DataFrame

,age,sex,height,weight,nurse,site,device,recording_date,report,scp_codes,...,burst_noise,electrodes_problems,extra_beats,strat_fold,filename_lr,filename_hr,scp_keys,has_MI,has_CD,has_ISCA
0,-0.227271,1,-0.036814,-0.711564,2.0,0.0,9,1984-11-09 09:17:34,sinusrhythmus periphere niederspannung,"{'NORM': 100.0, 'LVOLT': 0.0, 'SR': 0.0}",...,NaN,NaN,NaN,3,records100/00000/00001_lr,records500/00000/00001_hr,"[NORM, LVOLT, SR]",False,False,False
1,-2.414198,0,-0.036814,-0.041247,2.0,0.0,9,1984-11-14 12:55:37,sinusbradykardie sonst normales ekg,"{'NORM': 80.0, 'SBRAD': 0.0}",...,NaN,NaN,NaN,2,records100/00000/00002_lr,records500/00000/00002_hr,"[NORM, SBRAD]",False,False,False
2,-1.350287,1,-0.036814,-0.137007,2.0,0.0,9,1984-11-15 12:49:10,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",...,NaN,NaN,NaN,5,records100/00000/00003_lr,records500/00000/00003_hr,"[NORM, SR]",False,False,False
3,-2.118667,0,-0.036814,1.107868,2.0,0.0,9,1984-11-15 13:44:57,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",...,NaN,NaN,NaN,3,records100/00000/00004_lr,records500/00000/00004_hr,"[NORM, SR]",False,False,False
4,-2.414198,1,-0.036814,-0.041247,2.0,0.0,9,1984-11-17 10:43:15,sinusrhythmus normales ekg,"{'NORM': 100.0, 'SR': 0.0}",...,NaN,NaN,NaN,4,records100/00000/00005_lr,records500/00000/00005_hr,"[NORM, SR]",False,False,False


In [23]:
import pandas as pd

bf = pd.read_csv("ptbxl_cleaned_metadata.csv") 
bf.head(5)  # Display the first few rows of the cleaned DataFrame

,ecg_id,patient_id,age,sex,height,weight,site,device,heart_axis,infarction_stadium1,...,second_opinion,initial_autogenerated_report,validated_by_human,static_noise,pacemaker,strat_fold,scp_keys,has_MI,has_CD,has_ISCA
0,1,15709.0,-0.227271,1,-0.036814,-0.711564,0.0,CS-12 E,NaN,NaN,...,False,False,True,", I-V1,",NaN,3,"['NORM', 'LVOLT', 'SR']",False,False,False
1,2,13243.0,-2.414198,0,-0.036814,-0.041247,0.0,CS-12 E,NaN,NaN,...,False,False,True,NaN,NaN,2,"['NORM', 'SBRAD']",False,False,False
2,3,20372.0,-1.350287,1,-0.036814,-0.137007,0.0,CS-12 E,NaN,NaN,...,False,False,True,NaN,NaN,5,"['NORM', 'SR']",False,False,False
3,4,17014.0,-2.118667,0,-0.036814,1.107868,0.0,CS-12 E,NaN,NaN,...,False,False,True,NaN,NaN,3,"['NORM', 'SR']",False,False,False
4,5,17448.0,-2.414198,1,-0.036814,-0.041247,0.0,CS-12 E,NaN,NaN,...,False,False,True,NaN,NaN,4,"['NORM', 'SR']",False,False,False


In [24]:
 df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21837 entries, 0 to 21836
Data columns (total 27 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   age                           21837 non-null  float64
 1   sex                           21837 non-null  int64  
 2   height                        21837 non-null  float64
 3   weight                        21837 non-null  float64
 4   nurse                         20328 non-null  float64
 5   site                          21819 non-null  float64
 6   device                        21837 non-null  int8   
 7   recording_date                21837 non-null  object 
 8   report                        21837 non-null  object 
 9   scp_codes                     21837 non-null  object 
 10  infarction_stadium1           5626 non-null   object 
 11  validated_by                  12426 non-null  float64
 12  second_opinion                21837 non-null  bool   
 13  i